In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('2022_Competition_Training.csv')

/var/folders/_4/24x6m5pd4b95qt0xb6qk_6hr0000gn/T/ipykernel_32331/3381801404.py:1: DtypeWarning: Columns (760) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2022_Competition_Training.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48300 entries, 0 to 48299
Columns: 881 entries, id to hi_flag
dtypes: float64(817), int64(57), object(7)
memory usage: 324.6+ MB


In [3]:
# drop rows contain (*) in lang_spoken_cd , cms_race_cd , cms_ra_factor_type_cd

df = df[df['lang_spoken_cd'].isin(['*']) == False]
df = df[df['cms_race_cd'].isin(['*']) == False]
df = df[df['cms_ra_factor_type_cd'].isin(['*']) == False]

In [4]:
list = ['sex_cd', 'lang_spoken_cd', 'cms_orig_reas_entitle_cd', 'cms_race_cd', 'cons_mobplus', 'cons_homstat', 'rucc_category', 'cms_ra_factor_type_cd']
for i in list:
    data = df[i].value_counts()
    print(data, '\n')

F    29083
M    19179
Name: sex_cd, dtype: int64 

ENG    28267
SPA     1067
Name: lang_spoken_cd, dtype: int64 

0.0    33339
1.0    13017
3.0       28
2.0       12
Name: cms_orig_reas_entitle_cd, dtype: int64 

1    31943
2     6572
1     5586
2     1127
5      921
3      656
0      646
4      252
5      146
0      135
6      113
3      102
4       45
6       18
Name: cms_race_cd, dtype: int64 

M    21458
S    11641
U     2940
P     1088
Name: cons_mobplus, dtype: int64 

Y    26728
P     4265
U     3521
R     2413
T      191
Name: cons_homstat, dtype: int64 

1-Metro       21002
2-Metro       12916
3-Metro        5316
6-Nonmetro     3239
4-Nonmetro     2711
7-Nonmetro     1449
5-Nonmetro      832
8-Nonmetro      448
9-Nonmetro      349
Name: rucc_category, dtype: int64 

CN    37922
CP     3270
E      3096
CF     1302
D       169
C2       23
SE       20
Name: cms_ra_factor_type_cd, dtype: int64 



In [6]:
df[list].dtypes

sex_cd                       object
lang_spoken_cd               object
cms_orig_reas_entitle_cd    float64
cms_race_cd                  object
cons_mobplus                 object
cons_homstat                 object
rucc_category                object
cms_ra_factor_type_cd        object
dtype: object

In [5]:
# Get_dummies for 8 categorial features 

df = pd.get_dummies(df, columns=list)

In [6]:
# Drop columns 'id' and 'hi_flag', Replace all the NaN values with 0

new_df = df.drop(['id', 'hi_flag'],axis = 1)

# Replace the NaN values with median
for i in new_df.columns :
    new_df[i].fillna(new_df[i].median(), inplace=True)

/var/folders/_4/24x6m5pd4b95qt0xb6qk_6hr0000gn/T/ipykernel_32331/3785802792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[i].fillna(new_df[i].median(), inplace=True)
/var/folders/_4/24x6m5pd4b95qt0xb6qk_6hr0000gn/T/ipykernel_32331/3785802792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[i].fillna(new_df[i].median(), inplace=True)
/var/folders/_4/24x6m5pd4b95qt0xb6qk_6hr0000gn/T/ipykernel_32331/3785802792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [7]:
new_df = new_df.fillna(0)

In [8]:
# Standard scale all the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(new_df), columns = new_df.columns)

In [9]:
y = df['hi_flag']

In [10]:
print(X.shape, y.shape)

(48262, 918) (48262,)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=1/3,random_state=10)

### Fitting models on the training data

In [15]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rforest = RandomForestClassifier()
rforest.fit(X_train, y_train)  
pred = rforest.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     15373
           1       0.00      0.00      0.00       715

    accuracy                           0.96     16088
   macro avg       0.48      0.50      0.49     16088
weighted avg       0.91      0.96      0.93     16088

[[15373     0]
 [  715     0]]


/Users/vinhkhang20/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vinhkhang20/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vinhkhang20/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
pred = dtree.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

NameError: name 'metrics' is not defined

In [53]:
# KNeighborsClassifier(knn)
from sklearn.neighbors import KNeighborsClassifier  

knn= KNeighborsClassifier(n_neighbors=3)  
knn.fit(X_train, y_train)  
pred = knn.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     15373
           1       0.08      0.01      0.02       715

    accuracy                           0.95     16088
   macro avg       0.52      0.50      0.50     16088
weighted avg       0.92      0.95      0.93     16088

[[15301    72]
 [  709     6]]


In [12]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train,y_train)
pred= model.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.08      0.14     15373
           1       0.05      0.94      0.09       715

    accuracy                           0.12     16088
   macro avg       0.50      0.51      0.11     16088
weighted avg       0.92      0.12      0.14     16088

[[ 1186 14187]
 [   45   670]]


In [16]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)
pred = LR.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     15373
           1       0.16      0.01      0.02       715

    accuracy                           0.95     16088
   macro avg       0.56      0.50      0.50     16088
weighted avg       0.92      0.95      0.93     16088

[[15326    47]
 [  706     9]]


/Users/vinhkhang20/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
#Perceptron
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     15373
           1       0.09      0.08      0.08       715

    accuracy                           0.92     16088
   macro avg       0.52      0.52      0.52     16088
weighted avg       0.92      0.92      0.92     16088

[[14776   597]
 [  659    56]]


In [58]:
# SVM
from sklearn import svm
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

for i in kernel : 
    clf = svm.SVC(kernel=i) 
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print(metrics.classification_report(y_test, pred))
    print(metrics.confusion_matrix(y_test, pred))   

In [15]:
# Neural Network

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier()
clf.fit(X_train, y_train) 
pred = clf.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     15373
           1       0.13      0.05      0.07       715

    accuracy                           0.94     16088
   macro avg       0.55      0.52      0.52     16088
weighted avg       0.92      0.94      0.93     16088

[[15134   239]
 [  678    37]]


In [18]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
pred = gbc.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     15373
           1       0.05      0.00      0.00       715

    accuracy                           0.95     16088
   macro avg       0.50      0.50      0.49     16088
weighted avg       0.92      0.95      0.93     16088

[[15354    19]
 [  714     1]]


### Combining Random Oversampling and Undersampling

In [ ]:
pip install imblearn

In [115]:
print(X_train.shape, y_train.shape)

(32174, 918) (32174,)


In [158]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
# # define oversampling strategy
# over = RandomOverSampler(sampling_strategy=0.1)
# # # fit and apply the transform
# X_train, y_train = over.fit_resample(X_train, y_train)
# # define undersampling strategy
under = RandomUnderSampler(sampling_strategy=0.5)
# # fit and apply the transform
X_train, y_train = under.fit_resample(X_train, y_train)

In [159]:
print(X_train.shape, y_train.shape)

(4197, 918) (4197,)


In [161]:
gbc = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1.0, random_state=10, max_features=None,
warm_start=False)
gbc.fit(X_train, y_train)
pred = gbc.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93     15373
           1       0.12      0.33      0.18       715

    accuracy                           0.86     16088
   macro avg       0.54      0.61      0.55     16088
weighted avg       0.93      0.86      0.89     16088

[[13674  1699]
 [  477   238]]


In [43]:
# Fit through a Decision Tree
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
pred = dtree.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.71      0.82     15373
           1       0.07      0.43      0.11       715

    accuracy                           0.70     16088
   macro avg       0.51      0.57      0.47     16088
weighted avg       0.92      0.70      0.79     16088

[[10903  4470]
 [  404   311]]


In [44]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train,y_train)
pred = clf.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.74      0.84     15373
           1       0.08      0.49      0.14       715

    accuracy                           0.73     16088
   macro avg       0.53      0.62      0.49     16088
weighted avg       0.93      0.73      0.81     16088

[[11449  3924]
 [  364   351]]


In [129]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate=0.01, n_estimators=1200,max_depth=9, min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True)
gbc.fit(X_train, y_train)
pred = gbc.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93     15373
           1       0.13      0.34      0.19       715

    accuracy                           0.87     16088
   macro avg       0.55      0.62      0.56     16088
weighted avg       0.93      0.87      0.90     16088

[[13740  1633]
 [  471   244]]


### **Using Resample**

In [12]:
#Reset index and concat the train data
train_df = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)],axis = 1)

In [13]:
y_train.value_counts()

0    30775
1     1399
Name: hi_flag, dtype: int64

In [14]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = train_df[(train_df['hi_flag']==0)] 
df_minority = train_df[(train_df['hi_flag']==1)] 
# upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample with replacement
                                 n_samples= 40000, # to match majority class
                                 random_state=42)  # reproducible results
# Combine majority class with upsampled minority class
resample_df = pd.concat([df_minority_upsampled, df_majority])
resample_df['hi_flag'].value_counts()

1    40000
0    30775
Name: hi_flag, dtype: int64

In [15]:
X_train_new = resample_df.drop(['hi_flag'], axis = 1)
y_train_new = resample_df['hi_flag']

In [16]:
from sklearn import metrics

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(learning_rate=0.01, n_estimators=100,max_depth=100, min_samples_split=2, min_samples_leaf=1, subsample=1.0, random_state=10, max_features=None,
warm_start=False)
gbc.fit(X_train_new, y_train_new)
pred = gbc.predict(X_test)
print(metrics.classification_report(y_test, pred))
print(metrics.confusion_matrix(y_test, pred))

### Predict probability for the Holdout file

In [178]:
ho_df = pd.read_csv('2022_Competition_Holdout.csv')

/var/folders/_4/24x6m5pd4b95qt0xb6qk_6hr0000gn/T/ipykernel_54471/264395044.py:1: DtypeWarning: Columns (760) have mixed types. Specify dtype option on import or set low_memory=False.
  ho_df = pd.read_csv('2022_Competition_Holdout.csv')


In [179]:
# drop rows contain (*) in lang_spoken_cd , cms_race_cd , cms_ra_factor_type_cd
for i in ['lang_spoken_cd' , 'cms_race_cd' , 'cms_ra_factor_type_cd']:
    ho_df = ho_df[ho_df[i].isin(['*']) == False]

# Get_dummies for 8 categorial features 
ho_df = pd.get_dummies(ho_df, columns=list)

# Drop columns 'id', Replace all the NaN values with 0
new_ho_df = ho_df.drop(['id'],axis = 1).fillna(0)

# Standard scale all the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
new_ho_df = pd.DataFrame(sc.fit_transform(new_ho_df), columns = new_ho_df.columns)

In [61]:
gbc.classes_

array([0, 1])

In [180]:
y_prob = gbc.predict_proba(new_ho_df)
ho_df['id'].shape

(12210,)

In [181]:
y_prob.shape

(12210, 2)

In [183]:
score_rank = pd.concat([ho_df['id'].reset_index(drop=True), pd.DataFrame(y_prob, columns =['hi_0', 'hi_1']).reset_index(drop=True)], axis=1)
score_rank

,id,hi_0,hi_1
0,100093066.0,0.827840,0.172160
1,100313000.0,0.394708,0.605292
2,100330875.0,0.705304,0.294696
3,100346385.0,0.711053,0.288947
4,100443164.0,0.547432,0.452568
...,...,...,...
12205,999896529.0,0.696731,0.303269
12206,999899065.0,0.383501,0.616499
12207,999946210.0,0.730865,0.269135
12208,999984658.0,0.630996,0.369004


In [185]:
score_rank['rank'] = score_rank['hi_1'].rank(ascending=False, method = 'first')
score_rank

,id,hi_0,hi_1,rank
0,100093066.0,0.827840,0.172160,10936.0
1,100313000.0,0.394708,0.605292,1447.0
2,100330875.0,0.705304,0.294696,7547.0
3,100346385.0,0.711053,0.288947,7677.0
4,100443164.0,0.547432,0.452568,3918.0
...,...,...,...,...
12205,999896529.0,0.696731,0.303269,7322.0
12206,999899065.0,0.383501,0.616499,1303.0
12207,999946210.0,0.730865,0.269135,8214.0
12208,999984658.0,0.630996,0.369004,5700.0


In [186]:
output = score_rank[['id', 'hi_1', 'rank']]
output.columns = ['ID', 'SCORE', 'RANK']
output.to_csv('2022CaseCompetition_Yini_Rong_20221007.csv', sep=',')

In [187]:
score_rank[['id', 'hi_1', 'rank']]

,id,hi_1,rank
0,100093066.0,0.172160,10936.0
1,100313000.0,0.605292,1447.0
2,100330875.0,0.294696,7547.0
3,100346385.0,0.288947,7677.0
4,100443164.0,0.452568,3918.0
...,...,...,...
12205,999896529.0,0.303269,7322.0
12206,999899065.0,0.616499,1303.0
12207,999946210.0,0.269135,8214.0
12208,999984658.0,0.369004,5700.0


### Using LightGBM